In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 r

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

2025-05-27 12:10:25.625949: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748347825.822557      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748347825.875862      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
model_name = "Qwen/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [4]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Tokenizer.pad_token diatur ke: {tokenizer.pad_token}")

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.float16 
)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [6]:
if model.config.pad_token_id is None or model.config.pad_token_id != tokenizer.pad_token_id:
    model.config.pad_token_id = tokenizer.pad_token_id
    print(f"Model.config.pad_token_id diatur ke: {model.config.pad_token_id}")

Model.config.pad_token_id diatur ke: 151643


In [7]:
lora_config = LoraConfig(
    r=32, # Coba tingkatkan r jika memungkinkan
    lora_alpha=64, # Biasanya 2x dari r
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.enable_input_require_grads()
model.print_trainable_parameters()

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


In [8]:
dataset = load_dataset("json", data_files="/kaggle/input/sampah-dataset/combined_all_datasets.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
MAX_LENGTH = 1024

In [10]:
def create_sft_datapoint(example):
    instruction = example['instruction']
    output_text = example['output']
    user_content = instruction # Karena example['input'] kosong

    messages = [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": output_text}
    ]
    
    full_text_str = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False
    )
    model_input = tokenizer(
        full_text_str,
        max_length=MAX_LENGTH,
        truncation=True,
        padding=False,
        return_attention_mask=True
    )
    input_ids = model_input['input_ids']
    attention_mask = model_input['attention_mask']
    labels = list(input_ids)

    prompt_messages = [{"role": "user", "content": user_content}]
    prompt_only_str = tokenizer.apply_chat_template(
        prompt_messages, tokenize=False, add_generation_prompt=True
    )
    
    tokenized_prompt_ids = tokenizer(
        prompt_only_str, max_length=MAX_LENGTH, truncation=True, add_special_tokens=False
    )['input_ids']
    prompt_length = len(tokenized_prompt_ids)

    for i in range(prompt_length):
        if i < len(labels):
            labels[i] = -100
        else:
            break
            
    if output_text and all(label == -100 for label in labels):
        print(f"Peringatan: Semua label di-mask untuk contoh: {instruction[:50]}... Prompt length: {prompt_length}, Input IDs length: {len(input_ids)}")
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [11]:
tokenized_dataset_train = dataset["train"].map(
    create_sft_datapoint,
    remove_columns=list(dataset["train"].features) # Hapus kolom asli
)

Map:   0%|          | 0/2529 [00:00<?, ? examples/s]

In [12]:
training_args = TrainingArguments(
    output_dir="./qwen-lora-sampah-t4x2",
    per_device_train_batch_size=1,  # ❗ Krusial untuk T4 7B. Mungkin bisa 2 jika MAX_LENGTH < 512.
    gradient_accumulation_steps=16, # Sesuaikan untuk mencapai batch size efektif (misal, 1*2*16 = 32)
    gradient_checkpointing=True,    # ❗ Sangat penting untuk hemat memori, walau training lebih lambat ~20-30%
    num_train_epochs=3,             # 1-3 epoch biasanya cukup untuk LoRA
    learning_rate=4e-5,             # ❗ Coba antara 2e-5 hingga 5e-5. 2e-4 terlalu tinggi.
    fp16=True,                      # Wajib untuk T4
    optim="paged_adamw_8bit",       # ❗ Hemat memori untuk optimizer dengan 4-bit model
    logging_steps=25,               # Sesuaikan berdasarkan total steps (misal, 10-50)
    save_strategy="steps",
    save_steps=50,                  # Simpan checkpoint secara berkala
    save_total_limit=2,             # Batasi jumlah checkpoint
    lr_scheduler_type="cosine",     # Pilihan scheduler yang baik
    warmup_ratio=0.03,              # Sedikit warmup untuk stabilisasi training
    report_to="none",               # Set ke "wandb" atau "tensorboard" jika Anda menggunakannya
    disable_tqdm=False,
    # evaluation_strategy="steps",    # Jika Anda punya eval dataset
    # eval_steps=50,                  # Sesuaikan dengan save_steps
    # load_best_model_at_end=True,    # Jika menggunakan evaluasi
)

In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train, # Gunakan dataset yang sudah diproses
    # eval_dataset=tokenized_dataset_eval, # Jika ada
    data_collator=data_collator
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
print("Memulai fine-tuning...")
trainer.train()
print("Fine-tuning selesai.")

Memulai fine-tuning...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
25,1.868100
50,1.339400
75,1.199300
100,1.106900
125,1.072500
150,1.042300
175,0.947100
200,0.878500
225,0.884500
250,0.848200


Fine-tuning selesai.


In [16]:
output_direktori_model = "./hasil_finetuning_qwen_chatbot" # Anda bisa ganti nama direktorinya

print(f"Menyimpan adapter LoRA model ke: {output_direktori_model}...")
model.save_pretrained(output_direktori_model)
print("Adapter LoRA berhasil disimpan.")

print(f"Menyimpan tokenizer ke: {output_direktori_model}...")
tokenizer.save_pretrained(output_direktori_model)
print("Tokenizer berhasil disimpan.")

Menyimpan adapter LoRA model ke: ./hasil_finetuning_qwen_chatbot...
Adapter LoRA berhasil disimpan.
Menyimpan tokenizer ke: ./hasil_finetuning_qwen_chatbot...
Tokenizer berhasil disimpan.


In [18]:
model.eval()
print("Model diatur ke mode evaluasi. Siap untuk diuji.")

# --- Fungsi untuk Chat Interaktif ---
# Riwayat percakapan disimpan dalam format yang diharapkan oleh tokenizer.apply_chat_template
messages_history = [] 

print("\n--- Chatbot Interaktif Dimulai (Ketik 'quit' atau 'exit' untuk keluar) ---")
while True:
    user_input = input("Anda: ")
    if user_input.lower() in ['quit', 'exit']:
        break

    current_turn_messages = list(messages_history) 
    current_turn_messages.append({"role": "user", "content": user_input})

    try:
        prompt_chat_template = tokenizer.apply_chat_template(
            current_turn_messages,
            tokenize=False,
            add_generation_prompt=True 
        )
    except Exception as e:
        print(f"Error saat menerapkan chat template: {e}")
        continue

    # Pindahkan input ke device yang sama dengan model Anda
    # Jika 'model' Anda menggunakan device_map="auto" atau sudah di .to(device) sebelumnya,
    # tokenizer().to(model.device) akan menempatkannya dengan benar.
    # Jika Anda tidak yakin, Anda bisa cek model.device
    device_to_use = model.device 
    inputs = tokenizer(prompt_chat_template, return_tensors="pt", padding=True).to(device_to_use)

    print("Chatbot sedang berpikir...")
    with torch.no_grad(): 
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=True,      
            temperature=0.6,     
            top_k=50,            
            top_p=0.95,          
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id 
        )
    
    response_ids = outputs[0][inputs.input_ids.shape[-1]:]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True).strip()
    
    print(f"Chatbot: {response_text}")

    messages_history.append({"role": "user", "content": user_input})
    messages_history.append({"role": "assistant", "content": response_text})

    if len(messages_history) > 10: 
        messages_history = messages_history[-10:]

print("Sesi interaktif selesai.")

Model diatur ke mode evaluasi. Siap untuk diuji.

--- Chatbot Interaktif Dimulai (Ketik 'quit' atau 'exit' untuk keluar) ---


Anda:  sebutkan jenis jenis sampah


Chatbot sedang berpikir...
Chatbot: Sampah umum, sampah B3, sampah anorganik, sampah organik, sampah kertas, sampah plastik, sampah logam, sampah elektronik, sampah fakta (sisa makanan). Sampah bisa dibagi berdasarkan sifatnya, jenis bahan, dan cara pengelolaannya. Pengelolaan yang tepat untuk setiap jenis sangat penting.


Anda:  exit


Sesi interaktif selesai.


In [19]:
output_direktori_untuk_deployment = "./qwen_chatbot_huggingface_space" # Ganti nama direktori sesuai keinginan

print(f"Menyimpan adapter LoRA model ke: {output_direktori_untuk_deployment}...")
# 'model' di sini adalah PeftModel Anda yang sudah dilatih
model.save_pretrained(output_direktori_untuk_deployment)
print("Adapter LoRA berhasil disimpan.")

print(f"Menyimpan tokenizer ke: {output_direktori_untuk_deployment}...")
tokenizer.save_pretrained(output_direktori_untuk_deployment)
print("Tokenizer berhasil disimpan.")

print(f"Model dan tokenizer siap untuk deployment disimpan di: {output_direktori_untuk_deployment}")

Menyimpan adapter LoRA model ke: ./qwen_chatbot_huggingface_space...
Adapter LoRA berhasil disimpan.
Menyimpan tokenizer ke: ./qwen_chatbot_huggingface_space...
Tokenizer berhasil disimpan.
Model dan tokenizer siap untuk deployment disimpan di: ./qwen_chatbot_huggingface_space


In [20]:
!zip -r /kaggle/working/qwen_chatbot_huggingface_space.zip /kaggle/working/qwen_chatbot_huggingface_space

  adding: kaggle/working/qwen_chatbot_huggingface_space/ (stored 0%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/adapter_config.json (deflated 56%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/merges.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 57%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/added_tokens.json (deflated 36%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/tokenizer_config.json (deflated 65%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/tokenizer.json (deflated 81%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/README.md (deflated 66%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/vocab.json (deflated 61%)
  adding: kaggle/working/qwen_chatbot_huggingface_space/special_tokens_map.json (deflated 61%)
